<a href="https://colab.research.google.com/github/rootofdata/kostat_AI_contest/blob/main/%5BFinal%5DEDA_%EB%B0%8F_embeding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pylab as plt
import logging

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Data

In [ ]:
data= pd.read_csv('/content/drive/MyDrive/통계청_AI경진대회/실습용 자료/1. 실습용자료.txt', sep ='|',encoding='CP949', header=0)
submission= pd.read_csv('/content/drive/MyDrive/통계청_AI경진대회/모델 개발용 자료/2. 모델개발용자료.txt', sep ='|',encoding='CP949', header=0)
ks = pd.read_csv('/content/drive/MyDrive/통계청_AI경진대회/한국표준산업분류(10차)_국문_test.csv',encoding='CP949',header=2)

In [ ]:
data.head()

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육


# Make dataframe for 한국표준산업분류(10차)_국문

In [ ]:
data.head()

In [ ]:
ks['코드.3']=ks['코드.3']//10
input = ks[['코드.3','항목명.3']].dropna()
input = input.reset_index(drop=True)
input['코드.3']=input['코드.3'].astype(int)
input['항목명.3']=input['항목명.3'].str.rstrip('업')
input.rename(columns={'코드.3':'digit_3','항목명.3':'NLP'},inplace=True)
a1=[]
for i in range(len(input)):
  a1.append('id_'+str(i+1000001))
input.insert(0,'AI_id',a1)

In [ ]:
col = ['text_obj','text_mthd','text_deal']
NLP = data[col].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df = data[['AI_id','digit_3']].copy()
df['NLP'] = NLP
df_concat=pd.concat([df,input])
df_concat['NLP']=df_concat['NLP'].astype(str)
df_concat.reset_index(inplace=True)
NLP = df_concat['NLP']

In [ ]:
# 텍스트 정제 함수 : 한글 이외의 문자는 전부 제거합니다.
import re
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글의 정규표현식을 나타냅니다.
    result = hangul.sub('', text)
    return result

for i in NLP:
  text_cleaning(i)

from nltk import Text
kolaw1 = Text(NLP, name="kolaw")

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 448 kB 60.4 MB/s 


In [ ]:
type(NLP)

pandas.core.series.Series

In [ ]:
from konlpy.tag import *
okt = Okt()
tokenized_okt = []
for i in range(len(NLP)):
  tokenized_okt.append(okt.nouns(NLP[i]))

In [ ]:
okt_copy= tokenized_okt.copy()

# Split 8글자 ,7글자 ,6글자 ,5글자 ,4글자

In [ ]:
count={}
for j in range(len(okt_copy)):
  for i in okt_copy[j]:
      try: count[i] += 1
      except: count[i]=1
bigword8 = {key: value for key, value in count.items() if len(key)==8 and value <50}
bw8= list(bigword8)
bw8_copy=bw8.copy()
def split8(data_name,i):
  data_name =bw8.copy()
  for k in range(len(bw8)):
    d=[]
    d.append(data_name[k][0:i])
    d.append(data_name[k][i:8])
    data_name[k]=d
  return data_name
b8_2=[]
b8_4=[]
b8_6=[]
b8_2=split8(b8_2,2)
b8_4=split8(b8_4,4)
b8_6=split8(b8_6,6)
for i in range(len(bw8_copy)):
  for j in range(len(okt_copy)):
    for item in okt_copy[j]:
      if item ==bw8_copy[i]:
         okt_copy[j].remove(bw8_copy[i])
         okt_copy[j].append(b8_2[i][0])
         okt_copy[j].append(b8_2[i][1])
         okt_copy[j].append(b8_4[i][0])
         okt_copy[j].append(b8_4[i][1])    
         okt_copy[j].append(b8_6[i][0])
         okt_copy[j].append(b8_6[i][1]) 

In [ ]:
count={}
for j in range(len(okt_copy)):
  for i in okt_copy[j]:
      try: count[i] += 1
      except: count[i]=1
bigword7 = {key: value for key, value in count.items() if len(key)==7 and value <50}
bw7= list(bigword7)
bw7_copy=bw7.copy()
def split7(data_name,i):
  data_name =bw7.copy()
  for k in range(len(bw7)):
    d=[]
    d.append(data_name[k][0:i])
    d.append(data_name[k][i:7])
    data_name[k]=d
  return data_name
b7_2=[]
b7_4=[]
b7_2=split7(b7_2,2)
b7_4=split7(b7_4,4)
for i in range(len(bw7_copy)):
  for j in range(len(okt_copy)):
    for item in okt_copy[j]:
      if item ==bw7_copy[i]:
         okt_copy[j].remove(bw7_copy[i])
         okt_copy[j].append(b7_2[i][0])
         okt_copy[j].append(b7_2[i][1])
         okt_copy[j].append(b7_4[i][0])
         okt_copy[j].append(b7_4[i][1])     

In [ ]:
count={}
for j in range(len(okt_copy)):
  for i in okt_copy[j]:
      try: count[i] += 1
      except: count[i]=1
bigword6 = {key: value for key, value in count.items() if len(key)==6 and value <50}
bw6= list(bigword6)
bw6_copy=bw6.copy()
def split6(data_name,i):
  data_name =bw6.copy()
  for k in range(len(bw6)):
    d=[]
    d.append(data_name[k][0:i])
    d.append(data_name[k][i:6])
    data_name[k]=d
  return data_name
b6_2=[]
b6_3=[]
b6_4=[]
b6_2=split6(b6_2,2)
b6_3=split6(b6_3,3)
b6_4=split6(b6_4,4)
for i in range(len(bw6_copy)):
  for j in range(len(okt_copy)):
    for item in okt_copy[j]:
      if item ==bw6_copy[i]:
         okt_copy[j].remove(bw6_copy[i])
         okt_copy[j].append(b6_2[i][0])
         okt_copy[j].append(b6_2[i][1])
         okt_copy[j].append(b6_3[i][0])
         okt_copy[j].append(b6_3[i][1])
         okt_copy[j].append(b6_4[i][0])
         okt_copy[j].append(b6_4[i][1])     

In [ ]:
count={}
for j in range(len(okt_copy)):
  for i in okt_copy[j]:
      try: count[i] += 1
      except: count[i]=1
bigword5 = {key: value for key, value in count.items() if len(key)==5 and value <50}
bw5= list(bigword5)
bw5_copy=bw5.copy()
def split5(data_name,i):
  data_name =bw5.copy()
  for k in range(len(bw5)):
    d=[]
    d.append(data_name[k][0:i])
    d.append(data_name[k][i:5])
    data_name[k]=d
  return data_name
b5_2=[]
b5_3=[]
b5_2=split5(b5_2,2)
b5_3=split5(b5_3,3)
for i in range(len(bw5_copy)):
  for j in range(len(okt_copy)):
    for item in okt_copy[j]:
      if item ==bw5_copy[i]:
         okt_copy[j].remove(bw5_copy[i])
         okt_copy[j].append(b5_2[i][0])
         okt_copy[j].append(b5_2[i][1])
         okt_copy[j].append(b5_3[i][0])
         okt_copy[j].append(b5_3[i][1])

In [ ]:
count={}
for j in range(len(okt_copy)):
  for i in okt_copy[j]:
      try: count[i] += 1
      except: count[i]=1
bigword4 = {key: value for key, value in count.items() if len(key)==4 and value <50}
bw4= list(bigword4)
bw4_copy=bw4.copy()
def split4(data_name,i):
  data_name =bw4.copy()
  for k in range(len(bw4)):
    d=[]
    d.append(data_name[k][0:i])
    d.append(data_name[k][i:4])
    data_name[k]=d
  return data_name
b4_1=[]
b4_2=[]
b4_3=[]
b4_1=split4(b4_1,1)
b4_2=split4(b4_2,2)
b4_3=split4(b4_3,3)
for i in range(len(bw4_copy)):
  for j in range(len(okt_copy)):
    for item in okt_copy[j]:
      if item ==bw4_copy[i]:
        okt_copy[j].append(b4_1[i][0])
        okt_copy[j].append(b4_1[i][1])
        okt_copy[j].append(b4_2[i][0])
        okt_copy[j].append(b4_2[i][1])
        okt_copy[j].append(b4_3[i][0])
        okt_copy[j].append(b4_3[i][1])  
      else: continue

In [ ]:
bigword_split_okt=okt_copy.copy()

# 불용어 처리

In [ ]:
import pandas as pd
stop_words = pd.read_csv('/content/drive/MyDrive/통계청_AI경진대회/한국어불용어100.txt',header=None, encoding ='utf-8',sep='\t')
stop_words = stop_words[0]

In [ ]:
for i in range(len(stop_words)):
  for j in range(len(bigword_split_okt)):
    for item in bigword_split_okt[j]:
      if item ==stop_words[i]:
          bigword_split_okt[j].remove(stop_words[i])

In [ ]:
search=['매장','대상','서비스','고객','시설','접객','사업','일반인','판매','요청','제공','중개','영업','일반','제조','및','등','업','용','해','류','위','도','집','타','통해','입','물','전','기타',
        '내','미만','상대로','스','로부터','위해','실','재','초','비','품','기','사','임','절','학','관','인','통해','캐','후','제','함','내부','즙','를','유','리','의','트','산','골','손','틀','접',
        '영','활','무','폐','드','다른','케','목','난','수','상','고', '사업장', '사무실', '가게', '매장', '영업장', '현장', '시설', '개인','계약', '의뢰', '수련장', '가공', '관리', '위주', '산업', '개별','운영',
        '활동', '요구', '업체', '음', '제작', '용품', '점', '가지','류', '각종', '의거', '대리', '체결', '설치', '방', '재료', '알선', '센터', '통해','전문', '수집', '기기', '업무', '부분', '실내', '특정', '서비스업',
        '단체', '구입', '온라인','소비자','사용자','사무소']
for i in range(len(search)):
  for j in range(len(bigword_split_okt)):
    for item in bigword_split_okt[j]:
      if item ==search[i]:
          bigword_split_okt[j].remove(search[i])

In [ ]:
remove_set =['카','철','쌀','떡','술','빵','집','차','닭','원','꽃','옷','펌','톱','쇠','소','콩','훍','약','은','중','즙','땅','풀','밥','면','회','찜','배','산','벼','금','숯']
bw = [i for i in bigword_split_okt if i not in remove_set]

In [ ]:
for i in range(len(bw)):
  for j in range(len(bigword_split_okt)):
    for item in bigword_split_okt[j]:
      if item ==bw[i]:
          bigword_split_okt[j].remove(bw[i])

for r, row in enumerate(bigword_split_okt):
  for c, letter in enumerate(row):
    if letter ==bw[r][c]:
          bigword_split_okt[r].remove(bw[r][c])

In [ ]:
import pickle
with open('/content/drive/MyDrive/통계청_AI경진대회/영석/Nyoungeeee_train','wb') as f:
    pickle.dump(bigword_split_okt,f)

In [ ]:
from gensim.models import Word2Vec
embedding_model = Word2Vec(bigword_split_okt, size=100, window = 2, min_count=50, workers=4, iter=100, sg=1)
embedding_model.save('/content/drive/MyDrive/통계청_AI경진대회/final/data/Nyoungeeee.model')